In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
def is_number(line):
    try: 
        int(line[0])
    except ValueError:
        return False
    return True

def is_msg(line):
    if line[0:3] == 'MSG':
        return True
    return False 

def is_newline(line):
    if line == '' or line == '\n':
        return True
    return False

def is_fix(line):
    if "SFIX" in line or "EFIX" in line:
        return True
    return False

def is_sacc(line):
    if "SSACC" in line or "ESACC" in line:
        return True
    return False

def is_blink(line):
    if "SBLINK" in line or "EBLINK" in line:
        return True
    return False    

In [12]:
def check_inpic(x, y, img_msg):
    [mid_x, mid_y, width, height] = img_msg[1:]
    start_x = float(mid_x) - float(width) / 2
    start_y = float(mid_y) - float(height) / 2
    end_x = float(mid_x) + float(width) / 2
    end_y = float(mid_y) + float(height) / 2
    try:
        x = float(x)
        y = float(y)
        if x >= start_x and x <= end_x and y >= start_y and y <= end_y:
            return True
        return False
    except:
        return False


def process_forcedmemory(msg_marker, msg, data, current_data_index, current_msg_index):
    assert("forcedmemory_start" in msg[current_msg_index][3])
    start_msg_index = current_msg_index
    event_name = msg[start_msg_index][3]
    img_msg = []
    start_time = msg[start_msg_index][2]
    end_time = -1
    while(1):
        current_msg_index += 1
        if "IMGLOAD CENTER" in msg[current_msg_index][3]:
            img_msg = msg[current_msg_index][3].split(" ")[3:] 
            # img_msg: [pic_loc, x, y, width, height]
        elif "forcedmemory_end" in msg[current_msg_index][3]:
            end_time = msg[current_msg_index][2]
            break

    while data[current_data_index][1] <= start_time:
        current_data_index += 1

    while data[current_data_index][1] <= end_time:
        msg_marker[current_data_index][0] = event_name
        current_x = data[current_data_index][2]
        current_y = data[current_data_index][3]
        if check_inpic(current_x, current_y, img_msg):
            msg_marker[current_data_index][1] = img_msg[0]
        else:
            msg_marker[current_data_index][1] = "not on pics"
        current_data_index += 1 
    return [current_data_index, current_msg_index, msg_marker]
 

def check_inbox(x, y, box_msg):
    [start_x, start_y, end_x, end_y] = box_msg
    start_x = float(start_x)
    start_y = float(start_y)
    end_x = float(end_x)
    end_y = float(end_y)
    try:
        x = float(x)
        y = float(y)
        if x >= start_x and x <= end_x and y >= start_y and y <= end_y:
            return True
        return False
    except:
        return False

def fix_on_picnum(x, y, img_msg_bottom, img_msg_left, img_msg_right):
    if check_inpic(x, y, img_msg_bottom):
        return 0
    elif check_inpic(x, y, img_msg_left):
        return 1
    elif check_inpic(x, y, img_msg_right):
        return 2
    else:
        return -1


def fix_on_boxnum(x, y, box_msg_bottom, box_msg_left, box_msg_right):
    if check_inbox(x, y, box_msg_bottom):
        return 0
    elif check_inbox(x, y, box_msg_left):
        return 1
    elif check_inbox(x, y, box_msg_right):
        return 2
    else:
        return -1


def process_forcedchoice(msg_marker, msg, data, current_data_index, current_msg_index):
    assert("forcedchoice_start" in msg[current_msg_index][3])
    start_msg_index = current_msg_index
    event_name = msg[start_msg_index][3]
    img_msgs = []
    box_msgs = []
    start_time = msg[start_msg_index][2]
    end_time = -1
    while(1):
        current_msg_index += 1
        if "IMGLOAD CENTER" in msg[current_msg_index][3]:
            img_msgs.append(msg[current_msg_index][3].split(" ")[3:])
            # img_msg: [pic_loc, x, y, width, height]
        elif "DRAWBOX" in msg[current_msg_index][3]:
            box_msgs.append(msg[current_msg_index][3].split(" ")[5:])
            # img_msg: [x, y, width, height]
        elif "forcedchoice_end" in msg[current_msg_index][3]:
            end_time = msg[current_msg_index][2]
            break
    
    while data[current_data_index][1] <= start_time:
        current_data_index += 1

    while data[current_data_index][1] <= end_time:
        msg_marker[current_data_index][0] = event_name
        current_x = data[current_data_index][2]
        current_y = data[current_data_index][3]
        fix_pic_num = fix_on_picnum(current_x, current_y, img_msgs[0], img_msgs[1], img_msgs[2])
        fix_box_num = fix_on_boxnum(current_x, current_y, box_msgs[0], box_msgs[1], box_msgs[2])
        # 0: bottom, 1: left, 2: right, -1: none
        if data[current_data_index][-2] == 'fix':
            if fix_pic_num != -1:
                msg_marker[current_data_index][1] = img_msgs[fix_pic_num][0]
            elif fix_box_num != -1:
                if fix_box_num == 0:
                    msg_marker[current_data_index][1] = "box_bottom"
                elif fix_box_num == 1:
                    msg_marker[current_data_index][1] = "box_left"
                elif fix_box_num == 2:
                    msg_marker[current_data_index][1] = "box_right"
            else:
                msg_marker[current_data_index][1] = "out"
        current_data_index += 1 
    return [current_data_index, current_msg_index, msg_marker]

In [10]:
current_dir = '0301'

expect_dir = '030705/'
file_to_parse = 'J030705.asc'

In [74]:

for current_dir in list_day:
    for i in range(1, 6):
        expect_dir = f'C:/Users/River/23summer/dku/edf_monkey/Data/{monkey}/{current_dir}/{current_dir}0{i}/'
        print(expect_dir)
        file_to_parse = f'C:/Users/River/23summer/dku/edf_monkey/Data/{monkey}/{current_dir}/M_{current_dir}_ASC/J{current_dir}0{i}.asc'

        print("start parsing file %s"%file_to_parse)

C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/050601/
start parsing file C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/M_0506_ASC/J050601.asc
C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/050602/
start parsing file C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/M_0506_ASC/J050602.asc
C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/050603/
start parsing file C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/M_0506_ASC/J050603.asc
C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/050604/
start parsing file C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/M_0506_ASC/J050604.asc
C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/050605/
start parsing file C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0506/M_0506_ASC/J050605.asc
C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0507/050701/
start parsing file C:/Users/River/23summer/dku/edf_monkey/Data/Mercury/0507/M_0507_ASC/J050701.asc
C:/Users/River/2

In [18]:
# Start matching marker and data
list_day = ['0301']

for current_dir in list_day:
    for i in range(1, 6):
        expect_dir = f'{current_dir}/{current_dir}0{i}/'
        print(expect_dir)

        # headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
        # data_f = pd.read_csv(""+ expect_dir + 'data.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": float, "col3": object, "col4": object, "col5": float, "col6": float})
        # data = data_f.to_numpy()
        # # Fixation
        # headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9']
        # fix_f = pd.read_csv(""+ expect_dir + 'efix.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": object, "col3": object, "col4": float, "col5": float, "col6": float, "col7": float, "col8": float, "col9": float})
        # fix_marker = fix_f.to_numpy()

        # # Blink
        # headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
        # blink_f = pd.read_csv(""+ expect_dir + 'eblink.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": object, "col3": object, "col4": float, "col5": float, "col6": float})
        # blink_marker = blink_f.to_numpy()

        # #sacc
        # headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'col11', 'col12']
        # sacc_f = pd.read_csv(""+ expect_dir + 'esacc.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": object, "col3": object, "col4": float, "col5": float, "col6": float, "col7": float, "col8": float, "col9": float, "col10": float, "col11": float, "col12": float})
        # sacc_marker = sacc_f.to_numpy()
        # data_marker = np.empty((np.shape(data)[0], 2), dtype=object)
        # data_marker[:] = ""

        # fix_index = 0
        # blink_index = 0
        # sacc_index = 0

        # for i in range(np.shape(data)[0]):
        #     record = data[i]
        #     current_time = record[1]
        #     if fix_index < len(fix_marker) and fix_marker[fix_index][4] < current_time:
        #         fix_index += 1
        #     if blink_index < len(blink_marker) and blink_marker[blink_index][4] < current_time:
        #         blink_index += 1
        #     if sacc_index < len(sacc_marker) and sacc_marker[sacc_index][4] < current_time:
        #         sacc_index += 1

        #     if fix_index < len(fix_marker) and current_time >= fix_marker[fix_index][3] and current_time <= fix_marker[fix_index][4]:
        #         data_marker[i] = ["fix", fix_marker[fix_index][0]]
        #     elif blink_index < len(blink_marker) and current_time >= blink_marker[blink_index][3] and current_time <= blink_marker[blink_index][4]:
        #         data_marker[i] = ["blink", blink_marker[blink_index][0]]
        #     elif sacc_index < len(sacc_marker) and current_time >= sacc_marker[sacc_index][3] and current_time <= sacc_marker[sacc_index][4]:
        #         data_marker[i] = ["sacc", sacc_marker[sacc_index][0]]
        #     else:
        #         continue
        # new_data = np.concatenate((data, data_marker), axis = 1)
        # pd.DataFrame(new_data).to_csv(""+ expect_dir + 'match_data.csv', index = False, header  = ['Index', 'Time','x','y','pupil size','CR', 'event marker', 'event index'])
        # Start matching coordinate and picture data
        import pandas as pd
        headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8']
        data_f = pd.read_csv(""+ expect_dir + 'match_data.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": float, "col3": object, "col4": object, "col5": float, "col6": float, "col7": object, "col8": object})
        data = data_f.to_numpy()
        headers = ['col1', 'col2', 'col3', 'col4']
        msg_f = pd.read_csv(""+ expect_dir + 'msg.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": object, "col3": float, "col4": object})
        msg = msg_f.to_numpy()
        msg_marker = np.empty((np.shape(data)[0], 2), dtype=object)
        msg_marker[:] = ""


        current_msg_index = 0
        current_data_index = 0
        while current_msg_index < np.shape(msg)[0]:
            if "forcedchoice_start" in msg[current_msg_index][3]:
                [current_data_index, current_msg_index, msg_marker] = process_forcedchoice(msg_marker, msg, data, current_data_index, current_msg_index)
            elif "forcedmemory_start" in msg[current_msg_index][3]:
                [current_data_index, current_msg_index, msg_marker] = process_forcedmemory(msg_marker, msg, data, current_data_index, current_msg_index)
            current_msg_index += 1
        pd.DataFrame(msg_marker).to_csv(""+ expect_dir + 'msg_marker.csv', index = True, header  = ['event', 'fix'])
        matched_data_pic = np.concatenate((data, msg_marker), axis = 1)
        pd.DataFrame(matched_data_pic).to_csv(""+ expect_dir + 'final_data.csv', index = False, header  = ['Index', 'Time','x','y','pupil size','CR', 'movement_marker', 'movement_index', 'pic_marker', 'behavior'])


0301/030101/
0301/030102/
0301/030103/
0301/030104/
0301/030105/


In [31]:
path = "0224/022402/esacc.csv"
headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'col11', 'col12']
sacc_f = pd.read_csv(path, sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": object, "col3": object, "col4": float, "col5": float, "col6": float, "col7": float, "col8": float, "col9": float, "col10": float, "col11": float, "col12": float})
sacc_marker = sacc_f.to_numpy()
print(type(sacc_marker))

<class 'numpy.ndarray'>
